In [1]:
library(forecast)
library(rminer)
library(rjson)
library(writexl)

source("functions.R")

Warning message:
"package 'forecast' was built under R version 4.1.3"
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Warning message:
"package 'rminer' was built under R version 4.1.3"
Warning message:
"package 'writexl' was built under R version 4.1.3"


In [2]:
df = read.csv("../data/sanitizedData.csv", header = TRUE, sep=",")[2:12]

In [3]:
K=7
Test = K
L=nrow(df)
NTS=K # number of predictions
H=NTS # from 1 to H ahead predictions

# this time series object only includes TRAIN (older) data:
LTR=L-H  

Runs = 7
S = 7
W=(L-Test)-(Runs-1)*S

In [4]:
getResultsForecast <- function(df,modelsForecast, target, window) {
    
    if (target == "all") TS = df$all
    if (target == "male") TS = df$male
    if (target == "female") TS = df$female
    if (target == "adult") TS = df$adult
    if (target == "young") TS = df$young
    
        
    YR=diff(range(TS))
    
    #add <- list()
    model_name <- c()
    model_MAE <- c()
    model_NMAE <- c()
    model_RMSE <- c()
    model_RRSE <- c()
    model_training_time <- c()
    
    
    for (i in 1:length(modelsForecast)){
        
        if(window == "none"){
            HD=holdout(TS,ratio=Test,mode="order") 

            dtr=ts(TS[HD$tr],frequency=K)

            start_time = Sys.time()

            if (modelsForecast[[i]] == "Arima"){
                M=suppressWarnings(auto.arima(dtr))
            }
            if (modelsForecast[[i]] == "Holt Winters"){
                M=suppressWarnings(HoltWinters(dtr)) 
            } 
            if (modelsForecast[[i]] == "NN") {
                M=suppressWarnings(nnetar(dtr))
            } 
            if (modelsForecast[[i]] == "ETS") {
                M=suppressWarnings(ets(dtr))
            }

            end_time = Sys.time()
            training_time <- as.numeric(difftime(end_time, start_time))
            
            
            
            # forecast docs (https://www.rdocumentation.org/packages/forecast/versions/8.16/topics/forecast)  
            predArr=forecast(M,h=length(HD$ts))$mean[1:Test] 
            
            Y <- TS[HD$ts]
                
            # save results
            MAE=mmetric(y=Y,x=predArr,metric="MAE",val=YR)
            NMAE=mmetric(y=Y,x=predArr,metric="NMAE",val=YR)
            RMSE=mmetric(y=Y,x=predArr,metric="RMSE",val=YR)
            RRSE=mmetric(y=Y,x=predArr,metric="RRSE",val=YR) 
            
            
        } else {
            # vectors for saving the results
            arr_MAE=vector(length=Runs)
            arr_NMAE=vector(length=Runs)
            arr_RMSE=vector(length=Runs)
            arr_RRSE=vector(length=Runs)
            arr_time=vector(length=Runs)
        
            for(b in 1:Runs){

                H=holdout(TS,ratio=Test,mode=window,iter=b,window=W,increment=S)   


                dtr=ts(TS[H$tr],frequency=K)

                start_time = Sys.time()
                
                if (modelsForecast[[i]] == "Arima"){
                    M=suppressWarnings(auto.arima(dtr))
                }
                if (modelsForecast[[i]] == "Holt Winters"){
                    M=suppressWarnings(HoltWinters(dtr)) 
                } 
                if (modelsForecast[[i]] == "NN") {
                    M=suppressWarnings(nnetar(dtr))
                } 
                if (modelsForecast[[i]] == "ETS") {
                    M=suppressWarnings(ets(dtr))
                }
                
                end_time = Sys.time()
                time <- as.numeric(difftime(end_time, start_time))

                # forecast docs (https://www.rdocumentation.org/packages/forecast/versions/8.16/topics/forecast)  
                predArr=forecast(M,h=length(H$ts))$mean[1:Test] # multi-step ahead forecasts (https://search.r-project.org/CRAN/refmans/forecast/html/meanf.html)

                # mmetric docs (https://www.rdocumentation.org/packages/rminer/versions/1.4.6/topics/mmetric)
                #ev[b]=mmetric(y=TS[H$ts],x=Pred,metric="MAE",val=YR)


                Y <- TS[H$ts]

                # save results
                arr_MAE[b]=mmetric(y=Y,x=predArr,metric="MAE",val=YR)
                arr_NMAE[b]=mmetric(y=Y,x=predArr,metric="NMAE",val=YR)
                arr_RMSE[b]=mmetric(y=Y,x=predArr,metric="RMSE",val=YR)
                arr_RRSE[b]=mmetric(y=Y,x=predArr,metric="RRSE",val=YR)
                arr_time[b]=time
            }

            # results
            MAE = mean(arr_MAE)
            NMAE = mean(arr_NMAE)
            RMSE = mean(arr_RMSE)
            RRSE = mean(arr_RRSE)
            training_time = sum(arr_time)
        }
        
        
        
        model_name <- append(model_name, modelsForecast[[i]])
        model_MAE  <- append(model_MAE, MAE)
        model_NMAE <- append(model_NMAE, NMAE)
        model_RMSE <- append(model_RMSE, RMSE)
        model_RRSE <- append(model_RRSE, RRSE)
        training_time <- append(model_training_time, training_time)
    }
    # create data frame with all the results
    results <- data.frame(model=model_name,
                 MAE=model_MAE,
                 NMAE=model_NMAE,
                 RMSE=model_RMSE,
                 RRSE=model_RRSE,
                 training_time = training_time,
                 stringsAsFactors=FALSE)
    results
}

In [22]:
getResultsRminer <- function(df, models, target, window, case, path){
    
    if (target == "all") TS = df$all
    if (target == "male") TS = df$male
    if (target == "female") TS = df$female
    if (target == "adult") TS = df$adult
    if (target == "young") TS = df$young
    
    
    timelags = c(1:3, 5:7)
    D=CasesSeries(TS,timelags)
    W2=W-max(timelags)
    
    YR=diff(range(TS))
    
    #add <- list()
    model_name <- c()
    model_MAE <- c()
    model_NMAE <- c()
    model_RMSE <- c()
    model_RRSE <- c()
    model_training_time <- c()
    
    target_form = y~.
     
    if(file.exists(path)){
        models <- fromJSON(file=path)
    } else {
        models <- gridSearch(D, models, names(D), target_form)
        exportJSON <- toJSON(models, indent=4)
        write(exportJSON, path)
    }

    for(i in 1:length(models)){
        if (models[[i]]$rminer_best_search$mparheuristic) {
            searchh=list(search=mparheuristic(models[[i]]$rminer,models[[i]]$rminer_best_search$algorithm),method=c("holdoutorder",2/3))
        } else {
            searchh=models[[i]]$rminer_best_search$algorithm
        }

        if(window=="none"){
            H=holdout(D$y,ratio=Test,mode="order")
            
            start_time = Sys.time()
            
            M2=fit(target_form, D[H$tr,], model=models[[i]]$rminer, search = searchh) # create forecasting model
            
            end_time = Sys.time()
            training_time <- as.numeric(difftime(end_time, start_time))
            
            predArr=lforecast(M2,D,start=(length(H$tr)+1),Test) # multi-step ahead forecasts
            
            Y <- D[H$ts,]$y
            
            # save results
            MAE=mmetric(y=Y,x=predArr,metric="MAE",val=YR)
            NMAE=mmetric(y=Y,x=predArr,metric="NMAE",val=YR)
            RMSE=mmetric(y=Y,x=predArr,metric="RMSE",val=YR)
            RRSE=mmetric(y=Y,x=predArr,metric="RRSE",val=YR)
            
        } else {
            # vectors for saving the results
            arr_MAE=vector(length=Runs)
            arr_NMAE=vector(length=Runs)
            arr_RMSE=vector(length=Runs)
            arr_RRSE=vector(length=Runs)
            arr_time=vector(length=Runs)

            
            for(b in 1:Runs) {
                H2=holdout(D$y,ratio=Test,mode=window,iter=b,window=W2,increment=S)
                
                start_time = Sys.time()
                
                M2=fit(target_form, D[H2$tr,], model=models[[i]]$rminer, search = searchh) # create forecasting model

                end_time = Sys.time()
                time <- as.numeric(difftime(end_time, start_time))
                
                predArr=lforecast(M2,D,start=(length(H2$tr)+1),Test) # multi-step ahead forecasts

                Y <- D[H2$ts,]$y

                # save results
                arr_MAE[b]=mmetric(y=Y,x=predArr,metric="MAE",val=YR)
                arr_NMAE[b]=mmetric(y=Y,x=predArr,metric="NMAE",val=YR)
                arr_RMSE[b]=mmetric(y=Y,x=predArr,metric="RMSE",val=YR)
                arr_RRSE[b]=mmetric(y=Y,x=predArr,metric="RRSE",val=YR)
                arr_time[b]=time
            }
            
            
            # results
            MAE = mean(arr_MAE)
            NMAE = mean(arr_NMAE)
            RMSE = mean(arr_RMSE)
            RRSE = mean(arr_RRSE) 
            training_time = sum(arr_time)
        }
        
        model_name <- append(model_name, models[[i]]$name)
        model_MAE  <- append(model_MAE, MAE)
        model_NMAE <- append(model_NMAE, NMAE)
        model_RMSE <- append(model_RMSE, RMSE)
        model_RRSE <- append(model_RRSE, RRSE)
        training_time <- append(model_training_time, training_time)
    }
    # create data frame with all the results
    results <- data.frame(model=model_name,
                 MAE=model_MAE,
                 NMAE=model_NMAE,
                 RMSE=model_RMSE,
                 RRSE=model_RRSE,
                 training_time = training_time,
                 stringsAsFactors=FALSE)
    results
}

In [6]:
runCasesForecast <- function(target = "all"){
    arr_cases <- c("1", "4")
    arr_windows <- c("none", "rolling", "incremental")
    modelsForecast <- c("Arima", "Holt Winters", "NN", "ETS")
    results <- data.frame()
    for(i in 1:length(arr_cases)){
        #get case
        case <- arr_cases[i]
        case_df <- getCaseDf(df, case)
            
        for(n in 1:length(arr_windows)){
            window <- arr_windows[n]
            r <- getResultsForecast(df=case_df, modelsForecast=modelsForecast, target=target, window=window)
            r[['case']] <- case
            r[['window']] <- window
            
            results <- rbind(results, r)
        }
    }
    results$package <- "Forecast"
    results$training_time <- round(results$training_time, 5)
    
    col_order <- c("case", "window", "package",
            "model", "MAE", "NMAE", "RMSE", "RRSE", "training_time")
    results <- results[, col_order]
    results <- results[with(results, order(case, window, package, model)), ]
    rownames(results) <- 1:nrow(results)
    write_xlsx(results,paste0("models/results/univariate/univariate_package=forecast_target=",target,".xlsx"))
    results
}

In [12]:
results2 <- runCasesForecast("adult")

In [25]:
runCasesRminer <- function(target){
    arr_cases <- c("1","4")
    arr_windows <- c("none", "rolling", "incremental")
    results <- data.frame()
    for(i in 1:length(arr_cases)){
        #get case
        case <- arr_cases[i]
        case_df <- getCaseDf(df, case)
        
        path <- paste0("models/best_params/univariate/univariate_target=",target,"_case=",case,".json")
        
        models <- fromJSON(file="models/rminer_models.json")

        for(n in 1:length(arr_windows)){
            window <- arr_windows[n]
            r <- getResultsRminer(df=case_df, models=models, target=target, window=window, case=case, path=path)
            r[['case']] <- case
            r[['window']] <- window
            results <- rbind(results, r)
        }
    }
    results$package <- "Rminer"
    col_order <- c("case", "window", "package",
               "model", "MAE", "NMAE", "RMSE", "RRSE", "training_time")
    results <- results[, col_order]
    results <- results[with(results, order(case, window, model, package)), ]
    rownames(results) <- 1:nrow(results)
    write_xlsx(results,paste0("models/results/univariate/univariate_package=rminer_target=",target,".xlsx"))
    results
}

In [30]:
results <- runCasesRminer(target="adult")

[18:46:03] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[18:46:03] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[18:46:03] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[18:46:03] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[18:46:03] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[18:46:03] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[18:46:03] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[18:46:03] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg

In [ ]:
results[with(results, order(MAE)), ][1:20,]